Instalisasi

In [1]:
from penerjemah_manga.assets.ctd import model2annotations, model2annotation
from manga_ocr import MangaOcr
from PIL import Image
from tqdm import tqdm

In [2]:
import os
import cv2
import PIL.Image
import json
import numpy as np

In [ ]:
mocr = MangaOcr()

In [311]:
# Pengganti cv2_imshow di colab, klo mengunakan colab di hapus saja def ini
def cv2_imshow(img_file):

    # Tampilkan gambar menggunakan cv2.imshow
    cv2.imshow('Gambar', img_file)

    # Tunggu tombol keyboard ditekan dan tutup jendela setelah itu
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Definisikan fungsi untuk menggambar kotak pembatas
def draw_bounding_boxes(image, lines):
    cropped_images = []

    for line in lines:
        values = line.split(' ')
        x, y, width, height = map(float, values[1:])

        width_pixel = int(width * (image.shape[1])) + 20
        height_pixel = int(height * image.shape[0]) + 10
        x_pixel = int(x * (image.shape[1])) - int((width_pixel - 20) / 2) - 5
        y_pixel = int(y * image.shape[0]) - int((height_pixel - 10) / 2) - 10

        cv2.rectangle(image, (x_pixel, y_pixel), (x_pixel + width_pixel, y_pixel + height_pixel), (0, 255, 0), 2)

        cropped_image = image[y_pixel:y_pixel + height_pixel, x_pixel:x_pixel + width_pixel]
        cropped_images.append(cropped_image)

    return cropped_images

def draw_white_boxes(image, lines):
    for line in lines:
        values = line.split(' ')
        x, y, width, height = map(float, values[1:])

        width_pixel = int(width * (image.shape[1])) + 5
        height_pixel = int(height * image.shape[0])
        x_pixel = int(x * (image.shape[1])) - int(width_pixel / 2)
        y_pixel = int(y * image.shape[0]) - int(height_pixel / 2)

        cv2.rectangle(image, (x_pixel, y_pixel), (x_pixel + width_pixel, y_pixel + height_pixel), (255, 255, 255), -1)
        
def process_image(lokasi_image, lines):
    image_polos = cv2.imread(lokasi_image)
    draw_white_boxes(image_polos, lines)
    return image_polos

# Fungsi untuk menambahkan teks dengan memecah baris jika terlalu panjang
def add_text_multiline(image, text, x, y, max_width, max_height):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    font_color = (0, 0, 0)  # Warna teks (hitam)
    spas = 4

     # Fungsi untuk menentukan ukuran font yang sesuai
    def get_font_scale(text, max_width, max_height, font=cv2.FONT_HERSHEY_SIMPLEX, font_thickness=1):
      font_scale = 1
      while True:
          (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
          if text_w <= max_width and text_h <= max_height:
              break
          font_scale -= 0.1
      return font_scale

    # Membagi teks menjadi baris-baris yang sesuai dengan lebar maksimum
    words = text.split()
    tlines = ['']
    current_line = 0
    for word in words:
        test_line = tlines[current_line] + ' ' + word if tlines[current_line] else word
        (text_w, text_h), _ = cv2.getTextSize(test_line, font, font_scale, font_thickness)

        if text_w <= max_width:
            tlines[current_line] = test_line
        else:
            current_line += 1
            tlines.append(word)

    # Hitung tinggi total teks
    total_text_height = len(tlines) * (text_h + 4)  # Spasi antar baris: 2 piksel

    if total_text_height>max_height:
      font_scale -= 0.5
      font_thickness = 1
      spas=-1
    else:
      font_scale = 1
      font_thickness = 2
      spas=4

    # Tentukan posisi vertikal tengah
    y_centered = y + int((max_height - total_text_height) / 2) + int(text_h/2) +1

    # Menambahkan teks ke gambar
    for i, line in enumerate(tlines):
        text_size = cv2.getTextSize(line, font, font_scale, font_thickness)[0]
        text_x = int(x + (max_width - text_size[0]) / 2)
        text_y = y_centered + i * (text_h + spas)  # Spasi antar baris: 2 piksel

        cv2.putText(image, line, (text_x, text_y), font, font_scale, font_color, font_thickness, lineType=cv2.LINE_AA)
        
def process_and_add_text(image, lines, tjepang):
    # Di dalam loop:
    for idx, line in enumerate(lines):
        values = line.split(' ')
        x, y, width, height = map(float, values[1:])
        width_pixel = int(width * (image.shape[1])) + 5
        height_pixel = int(height * image.shape[0]) + 10
        x_pixel = int(x * (image.shape[1])) - int(width_pixel / 2)
        y_pixel = int(y * image.shape[0]) - int(height_pixel / 2)
        
        text = tjepang[idx]
        if text!="":
            cv2.rectangle(image, (x_pixel, y_pixel), (x_pixel + width_pixel, y_pixel + height_pixel), (255, 255, 255), -1)
            add_text_multiline(image, text, x_pixel, y_pixel, width_pixel, height_pixel)
        
    return image


Menjalankan Program

In [76]:
img_dir = r'penerjemah_manga/example'                              # can be dir list

In [394]:
img_file

'project/12.webp'

In [393]:
kordinat = model2annotation(model_path, img_file, save_dir, save_json=False)
kordinat

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.88s/it]


['1 0.8291479820627803 0.13468750000000002 0.04035874439461884 0.081875\n1 0.2336322869955157 0.1775 0.042152466367713 0.10500000000000001']

In [395]:
[1]

[1]

In [389]:
def save_and_translate(img_file, save_dir='hasil', tl_dir='terjemahan'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Mulai Menerjemahkan
    model_path = r'penerjemah_manga/assets/comictextdetector.pt'
    kordinat = model2annotation(model_path, img_file, save_dir, save_json=False)
    if kordinat != ['']:
        lines = kordinat[0].split('\n')

        #kordinat = kordinat[::-1]

        # memotong gambar
        image_kotakin = cv2.imread(img_file)
        cropped_images = draw_bounding_boxes(image_kotakin, lines)

        # Mengambil teks Jepang dari potongan gambar
        tjepang = {}
        for i, cropped_image in enumerate(tqdm(cropped_images, desc='Processing images')):
            cropped_image2 = np.array(cropped_image, dtype=np.uint8)
            if cropped_image2 is not None and cropped_image2.any():
                img = Image.fromarray(cropped_image2)
                text = mocr(img)
            else:
                text = ""
            tjepang[i] = text
            #print(f"{i}. '{text}'")

        return tjepang,lines
    else:
        tjepang=""
        lines=""
        return tjepang,lines

def plance_input_text(img_file, tjepang, lines, save_dir='hasil', tl_dir='terjemahan'):
    if tjepang!="":
        print("--------------------------------")
        print("Terjemahkan json berikut ini:")
        print(tjepang)        
        j1=len(tjepang)
        print(f"------------------------------")
        tjepang_input = input()
        tjepang = eval(tjepang_input)
        j2=len(tjepang)
        file_name = os.path.basename(img_file)
        #print(f"j1={j1} dan j2={j2}")
        if (j1==j2):
            image = cv2.imread(img_file)
            image = process_and_add_text(image, lines, tjepang)
            cv2.imwrite(os.path.join(tl_dir, file_name), image)
            print(f"---disimpan ke {file_name}")
        else:
            print(f"Jumlah array kurang! untuk {file_name}")
            
def plance_text(img_file, tjepang_input, lines, save_dir='hasil', tl_dir='terjemahan'):        
        file_name = os.path.basename(img_file)
        if tjepang_input!="":         
            tjepang = eval(tjepang_input)
            if len(tjepang) == len(lines):
                image = cv2.imread(img_file)
                image = process_and_add_text(image, lines, tjepang)
                cv2.imwrite(os.path.join(tl_dir, file_name), image)
                print(f"---disimpan ke {file_name}")
            else:
                print(f"---Jumlah kalimat tidak sesuai untuk {file_name}")
        else:
            print(f"Input kosong! atau jumlah text kurang untuk {file_name}")


In [396]:
# Contoh penggunaan:
img_file = 'project/12.webp'
bjepang,lines = save_and_translate(img_file)
plance_text(img_file,"{0:'tes'}",lines)

Processing images: 100%|█████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.02s/it]

---Jumlah kalimat tidak sesuai untuk 12.webp


In [390]:
plance_text(img_file,"{0:'tes'}",lines)

---Jumlah kalimat tidak sesuai untuk 12.webp


In [369]:
# Path folder yang berisi file gambar
folder_path = 'project'

bjepang={}
lines={} 

# Loop semua file gambar di dalam folder
for nomor, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.webp'):
        print(filename)
        img_file_path = os.path.join(folder_path, filename)
        bjepang[nomor],lines[nomor] = save_and_translate(img_file_path)

for nomor, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.webp'):
        img_file_path = os.path.join(folder_path, filename)
        plance_input_text(img_file_path,bjepang[nomor],lines[nomor]) 

01.webp


Processing images: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:10<00:00,  1.29s/it]


02.webp


Processing images: 100%|█████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.16s/it]


03.webp


...

---disimpan ke 16.webp
--------------------------------
Terjemahkan json berikut ini:
{0: '邪魔する奴はぶっ潰す', 1: '誰に与することもしない', 2: 'そんな風に俺達を', 3: '生み出したのは', 4: 'お前らだ', 5: '俺は俺の味方だ'}
------------------------------


 {   0: 'Siapa pun yang menghalangi akan dihancurkan.',   1: 'Tidak akan bersekutu dengan siapa pun.',   2: 'Kalian, yang',   3: 'menciptakan kita',   4: 'dengan cara seperti itu.',   5: 'Aku adalah sekutuku sendiri.' }


---disimpan ke 17.webp
--------------------------------
Terjemahkan json berikut ini:
{0: 'くっ．．．', 1: '日々輝．．．', 2: '幹部の理根と俺の理想が違うのなら', 3: 'テメェだって俺の敵だ', 4: '強欲上等', 5: '強欲上等', 6: '俺は悪の怪人', 7: '俺は悪の怪人', 8: '悪手だって取るさ', 9: '取るさ', 10: '邪魔する＞敵は全てぶそれが口の答えだ！！', 11: '全部思い通りにならなきゃ気がすまねェんだよ'}
------------------------------


 {   0: 'Kuh...',   1: 'Hibi Kagayaki...',   2: 'Jika tujuan Kuno dari para kader dan idealku berbeda,',   3: 'kau juga lawanku.',   4: 'Tamak adalah nomor satu.',   5: 'Tamak adalah nomor satu.',   6: 'Aku adalah monster jahat.',   7: 'Aku adalah monster jahat.',   8: 'Aku akan mengambil langkah jahat jika perlu.',   9: 'Aku akan mengambil langkah jahat jika perlu.',   10: 'Menghalangi > Semua musuh adalah semuanya. Itu adalah jawaban saya!!',   11: 'Semua harus sesuai keinginanku atau aku tidak akan merasa puas.' }


---disimpan ke 18.webp


In [318]:
for nomor, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.webp'):
        img_file_path = os.path.join(folder_path, filename)
        plance_input_text(img_file_path,bjepang[nomor],lines[nomor]) 

--------------------------------
Terjemahkan json berikut ini:
{0: '第２２話「バイエングが一般区」に表示されると、', 1: 'うわあああッ！！', 2: '春場加ぎ', 3: 'ここではそのまま見田！', 4: 'いつも親切事子！！', 5: '市府中攻撃が', 6: 'ふふふっ', 7: '健気だね', 8: 'でもそんなオモチャ効かないよ', 9: 'そこの彼から聞いてなかった？', 10: '啄因因路', 11: '．．．'}
------------------------------


 {   0: 'Bab 22 "Byeong di Wilayah Umum" muncul,',   1: 'Wow!!!',   2: 'Lapangan musim semi?',   3: 'Tinggal di sana saja!',   4: 'Selalu hal yang baik!',   5: 'Serangan Kota-Fu Chuo',   6: 'Fufufu',   7: 'Lucu, bukan?',   8: 'Tapi mainan seperti itu tidak akan berhasil.',   9: 'Tidakkah kamu mendengarnya dari dia di sana?',   10: 'Kutsudoinmichi',   11: '...' }


by001.jpg
--------------------------------
Terjemahkan json berikut ini:
{0: 'やっぱ化け物だよこの人．．．！', 1: '半信半疑だったけど', 2: '本当に来栖君の言う通りだった', 3: 'それでも', 4: 'こんなチート相手でも通用しうるプランを組んだ', 5: 'あと少しあいつをこの場に留められたら', 6: '鍵は手に入る！', 7: '止まってくれ！'}
------------------------------


  {   0: 'Ternyata dia benar-benar monster ini...',   1: 'Meskipun aku ragu-ragu sebelumnya,',   2: 'Kata-kata Kurasu-kun ternyata benar.',   3: 'Namun,',   4: 'Dia berhasil membuat rencana yang dapat digunakan bahkan melawan lawan sekuat itu.',   5: 'Jika kita bisa menahannya sebentar lagi di sini,',   6: 'Kita bisa mendapatkan kunci!',   7: 'Berhentilah!', }


by002.webp
--------------------------------
Terjemahkan json berikut ini:
{0: 'じゃあまり', 1: '一辺倒でつまらないなぁ', 2: '！', 3: '２日目終わるまであと何分だ！', 4: '飽きちゃった', 5: 'その場合、フラフライスの方の中に出来るようなので、', 6: 'まだ５分！'}
------------------------------


...

by003.jpg
--------------------------------
Terjemahkan json berikut ini:
{0: '！', 1: 'だよな', 2: 'それしかねェ！', 3: 'もうヤケだ！', 4: '．．．．．ッ', 5: '直接叩きます！', 6: 'ああ！'}
------------------------------


 {   0: '!',   1: 'Ya, tentu saja!',   2: 'Itu saja yang bisa kita lakukan!',   3: 'Saya sudah bosan!',   4: '....!',   5: 'Saya akan menyerang langsung!',   6: 'Aah!' }


by004.jpg
--------------------------------
Terjemahkan json berikut ini:
{0: 'くっ．．．', 1: 'はああッ！', 2: 'ナイファイ', 3: 'でもキミは．．．', 4: 'ダメだよ', 5: '！', 6: 'グハッ！', 7: '大和君！'}
------------------------------


 {   0: 'Kuh...',   1: 'Haaa!',   2: 'Naifai',   3: 'Tapi kamu...',   4: 'Tidak bisa, tahu!',   5: 'Guha!',   6: 'Yamato-kun!',7: }


SyntaxError: expression expected after dictionary key and ':' (<string>, line 1)

# Jika memerlukan kotak dialog kosongnya

In [ ]:
# image_polos = process_image(lokasi_image, lines)
# Tampilkan gambar
# cv2_imshow(image_polos)